In [1]:
from requests import get
import re
import os
import numpy as np
import math

def checkOKStatusRequest (response):
 if (response.status_code == 200): return True
 else: return False

APIKey = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjI4ZWZlOTZkNDk2ZjQ3ZmE5YjMzNWY5NDU3NWQyMzViIiwiaCI6Im11cm11cjEyOCJ9" #Public key

In [2]:

def getDownloadURL (fileName):
	print(f"Fetching file {fileName}")

	try:
		response = get(f"https://api.dataplatform.knmi.nl/open-data/v1/datasets/windgegevens/versions/1.0/files/{fileName}/url", headers={"Authorization": f"Bearer {APIKey}"})
	except:
		print("Can't fetch URL (error in syntax probably)")
		return

	if (checkOKStatusRequest(response) == False):
		print(f"Fetching URL failed with status code {response.status_code}, body response:")
		print(response.content)
		return

	if (response.content):
		lastModified = response.json()["lastModified"]
		print(f"Fetched temporary download URL, file last modified {lastModified}")
		return response.json()["temporaryDownloadUrl"]


# print(getDownloadURL(2023, "01", "19", "14", "20"))


In [3]:
def downloadFile(year, month):
	
	# All UTC dates and times
	fileNameRequest = f"kis_tow_{year}{month}.gz" 
	
	url = getDownloadURL(fileNameRequest)

	try:
		response = get(url)
	except:
		print("Can't fetch URL (error in syntax probably)")
		return

	if (checkOKStatusRequest(response) == False):
		print(f"Fetching URL failed with status code {response.status_code}, body response:")
		print(response.content)
		return

	contentDisposition = response.headers["content-disposition"]
	fileName = re.findall("filename=\"(.+)\"", contentDisposition)[0]
	path = f"compressed/{fileName}"

	try:
			if(os.path.exists(path)): os.remove(path)
			open(path, "wb").write(response.content)
	except:
		print(f"Saving file {path} failed")
		return

In [4]:
startYear = 2022
startMonth = "12"
#Include this month

stopYear = 2022
stopMonth = "12"
#Include this month


startYearMonthCombined = startYear + int(startMonth) / 12
stopYearMonthCombined = stopYear + int(stopMonth) / 12

numberOfFilesToDownload = int((stopYearMonthCombined - startYearMonthCombined) * 12)

# print(numberOfFilesToDownload)

for i in range(numberOfFilesToDownload + 1):
 yearMonthCombined = startYearMonthCombined + i * 1/12
 year = math.floor(yearMonthCombined)

 month = int(round(((yearMonthCombined - year) * 12), 0))
 if (month == 0): 
  month = 12
  year = year - 1
 if (month < 10): month = f"0{month}"

 # print(year, month)
 downloadFile(year, month)

Fetching file kis_tow_202212.gz
Fetched temporary download URL, file last modified 2023-02-13T02:48:32+00:00
